<a href="https://colab.research.google.com/github/pulkit8690/Topsis-For-Pretrained-Model-Text-Classification-/blob/main/Topsis_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
def topsis(df: pd.DataFrame, wts: np.ndarray, impact: np.ndarray) -> pd.DataFrame:
    mat = np.array(df.iloc[:, 1:])
    rows, cols = mat.shape


    for i in range(cols):
        temp = 0
        for j in range(rows):
            temp += mat[j][i] ** 2
        temp = temp**0.5
        wts[i] /= temp

    weightedNormalized = mat * wts

    idealBestWorst = []

    for i in range(cols):
        maxi = weightedNormalized[:, i].max()
        mini = weightedNormalized[:, i].min()
        idealBestWorst.append((maxi, mini) if impact[i] == 1 else (mini, maxi))
    topsisScore = []
    for i in range(rows):
        temp_p, temp_n = 0, 0
        for j in range(cols):
            temp_p += (weightedNormalized[i][j] - idealBestWorst[j][0]) ** 2
            temp_n += (weightedNormalized[i][j] - idealBestWorst[j][1]) ** 2
        temp_p, temp_n = temp_p**0.5, temp_n**0.5

        topsisScore.append(temp_n / (temp_p + temp_n))

    df["score"] = np.array(topsisScore).T
    df["rank"] = df["score"].rank(method="max", ascending=False)
    df["rank"] = df.astype({"rank": int})["rank"]
    return df



In [ ]:
df1 = pd.read_csv("Inputdata.csv",index_col=0)


In [ ]:
wts =np.array([10]*10)
impacts = np.array([1,1,1,1,1,1,1,1,-1,-1])

In [ ]:
newDF = topsis(df1,wts,impacts)

In [ ]:
newDF.head()

In [ ]:
newDF.to_csv('Output.csv')

In [ ]:
df_melted = pd.melt(newDF, id_vars=['Model'], value_vars=['Accuracy', 'Precision', 'Recall', 'F1 Score', 'ROC-AUC',
                                                       'Average Precision', 'Time (s)', 'Log Loss' ,'score'])

# Plot using Seaborn facet grid
g = sns.FacetGrid(df_melted, col="variable", col_wrap=3, height=4, sharey=False)
g.map(sns.barplot, "Model", "value", palette='viridis')
g.set_axis_labels("Model", "Value")
g.set_xticklabels(rotation=45, ha='right')
g.set_titles(col_template="{col_name}")
plt.tight_layout()
plt.savefig('parameter.png')
plt.show()

In [ ]:
plt.figure(figsize=(8,8))
plt.bar(newDF.Model,newDF.score,color = 'purple')
plt.xlabel('Model',labelpad=10)
plt.ylabel('TOPSIS Score')
plt.title('TOPSIS Score Comparison')
plt.xticks(rotation=90)
plt.tight_layout()
plt.savefig('comparison.png')
plt.show()